# O1v5-SONATA connectome manipulation preparation to add reciprocal connections
This is an auxiliary notebook for configuring model building and rewiring, which...
 - creates connectivity models from adjacency matrices containing additional reciprocal connections
 - sets up rewiring using these connectivity models to add reciprocal connections to the original connectome

<u>Requirements</u>: [connectome-manipulator](https://bbpgitlab.epfl.ch/conn/structural/connectome_manipulator) package (v0.0.10.dev1 or later)

ℹ️ Related ticket: [[ACCS-62](https://bbpteam.epfl.ch/project/issues/browse/ACCS-62)] "Adding reciprocal connections to V5 connectome"


In [1]:
# Initialization
import json
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle
from bluepysnap import Circuit
from connectome_manipulator.model_building import conn_prob_adj
from scipy import sparse as sps

Load and check adjacency matrix with connections to be added

In [36]:
# Specify adjacency matrices for adding connections
adj_seed = 0  # Random seed
adj_factor = 4  # Increase factor of RC's per dimension
adj_path = "/gpfs/bbp.cscs.ch/project/proj102/egas/reliability/data"

## Structured
# adj_file = os.path.join(adj_path, "mats_rc_on_simplices.pkl")
# adj_name = f"struct{adj_seed}_{adj_factor}x"
# check_rc = True  # Check that added connections are reciprocal

## Control
adj_file = os.path.join(adj_path, "mats_rc_on_simplices_controls.pkl")
adj_name = f"control{adj_seed}_{adj_factor}x"
check_rc = False  # Don't check that added connections are reciprocal

# Specify original adjacency (for checks)
orig_file = os.path.join(adj_path, "mats_rc_on_simplices.pkl")
orig_key = "original"

In [37]:
# Load original & structured/control adjacency matrices
with open(orig_file, "rb") as f:
    tmp_dict = pickle.load(f)
orig_adj = tmp_dict[orig_key]
print(f'Loaded "{orig_key}" adjacency matrix with {orig_adj.count_nonzero()} edges')

with open(adj_file, "rb") as f:
    tmp_dict = pickle.load(f)
adj = tmp_dict[f"modified_{adj_factor}"][adj_seed]
print(f'Loaded "{adj_name}" adjacency matrix with {adj.count_nonzero()} edges')

Loaded "original" adjacency matrix with 6717001 edges
Loaded "control0_4x" adjacency matrix with 7384380 edges


In [38]:
# Load circuit & node ids corresponding to adjacency matrix
circuit_path = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA'
circuit_name = os.path.split(circuit_path)[-1]
circuit_config = os.path.join(circuit_path, 'sonata', 'circuit_config_tmp.json') # SONATA config (.json)  # TEMP edges file with afferent_center_x/y/z properties preliminarily added w/o validation, see [NSETM-1222]

c = Circuit(circuit_config)
nodes = c.nodes["default"]
edges = c.edges["default"]

mc2_nodeset = "mc2_Column"
syn_sel = "EXC"
nids = np.intersect1d(nodes.ids(mc2_nodeset), nodes.ids({"synapse_class": syn_sel}))

assert adj.shape[0] == adj.shape[1] == len(nids), "ERROR: Nodes inconsistent with size of adjacency matrix!"
assert orig_adj.shape[0] == orig_adj.shape[1] == len(nids), "ERROR: Nodes inconsistent with size of original adjacency matrix!"

In [39]:
# Consistency check with original O1v5 adjacency matrices

## From structural comparator
with open("/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/struct_comparison/O1v5-SONATA/data/Orig_Adjacency_mc2EE.pickle", "rb") as f:
    tmp_dict = pickle.load(f)
tmp_adj = tmp_dict["adj"]["data"]
assert np.array_equal(orig_adj.toarray(), tmp_adj.toarray()), "ERROR: Adjacency mismatch (structural comparator)!"

## From baseline sims
tmp_adj = sps.load_npz("/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_Baseline/working_dir/connectivity.npz")
tmp_sel = np.isin(nodes.ids(mc2_nodeset), nids)
tmp_adj = tmp_adj[:, tmp_sel][tmp_sel, :]
assert np.array_equal(orig_adj.toarray(), tmp_adj.toarray()), "ERROR: Adjacency mismatch (baseline sims)!"

## From toposample paper (zenodo)
# tmp_adj = sps.load_npz("/gpfs/bbp.cscs.ch/project/proj83/home/pokorny/PLOS_ONE_Toposample_Paper/zenodo/input_data/connectivity.npz")
# tmp_sel = np.isin(nodes.ids(mc2_nodeset), nids)
# tmp_adj = tmp_adj[:, tmp_sel][tmp_sel, :]
# assert np.array_equal(orig_adj.toarray(), tmp_adj.toarray()), "ERROR: Adjacency mismatch (toposample)!"
# IGNORE: Adjacency has 6717002 edges for some reason; one additional edge is at index tmp_adj[26566, 1141]

## Also directly checked with bluepy (old v5 version), bluepysnap (SONATA version), and conntility (SONATA version): 6717001 edges

In [40]:
# Check edges to be added
assert np.all(adj[orig_adj.nonzero()]), "ERROR: Connections inconsistent with original adjacency matrix!"
diff_mat = adj - orig_adj
num_add = diff_mat.size
print(f'Connections to be added in "{adj_name}": {num_add}')

# Check reciprocity
if check_rc:
    assert not np.any(orig_adj[diff_mat.nonzero()]), "ERROR: Edges to be added already exist!"
    assert np.all(orig_adj.T[diff_mat.nonzero()]), "ERROR: Edges to be added are not reciprocal!"
    print("Checked reciprocity ... OK")

Connections to be added in "control0_4x": 667379


Create and save adjacency model for rewiring

In [41]:
adj_model_path = "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/models"

In [42]:
adj_model = conn_prob_adj.build(adj.tocsc(), nids, nids)
adj_model_name = f"adj_model_{adj_name}"
adj_model.save_model(adj_model_path, adj_model_name)
print(adj_model)
adj_model_file = os.path.join(adj_model_path, adj_model_name + ".json")
assert os.path.exists(adj_model_file), f'ERROR: Model file "{adj_model_file}" not saved!'
print(f'Model saved to "{adj_model_file}"')

ConnProbAdjModel
  <26567x26567 sparse matrix of type '<class 'numpy.bool_'>'
	with 7384380 stored elements in Compressed Sparse Column format>
Model saved to "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/models/adj_model_control0_4x.json"


Create rewiring config

In [43]:
output_base_path = '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits'
config_paths = ['../configs', '/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/configs']  # Write to local and proj9 folder

In [44]:
def default_manip_config(circuit_config, seed=3210, N_split=None):
    """Generates a default manipulation config dict w/o any specific manipulation."""
    manip_config = {}
    manip_config['circuit_config'] = circuit_config
    manip_config['seed'] = seed
    if N_split is not None:
        manip_config['N_split_nodes'] = N_split
    return manip_config

def add_manip_to_config(manip_config, name, prob_model_file, delay_model_file, props_model_file):
    """Adds a 'ConnAddRC' rewiring operation to the (v4) manipulation config dict (in-place)."""
    assert 'manip' not in manip_config, "ERROR: Manipulation operation already specified!"
    manip_config['manip'] = {'name': f'ConnAddRC_mc2EE_{name}',
                             'fcts': [{'source': 'conn_rewiring',
                                       'sel_src': {'hypercolumn': 2, 'synapse_class': 'EXC'},
                                       'sel_dest': {'hypercolumn': 2, 'synapse_class': 'EXC'},
                                       'syn_class': 'EXC',
                                       'keep_indegree': False,
                                       'reuse_conns': False,
                                       'keep_conns': True,
                                       'reuse_pos': True,
                                       'rewire_mode': 'add_only',
                                       'gen_method': 'randomize',
                                       'amount_pct': 100.0,
                                       'model_config': {'prob_model_spec': {'file': prob_model_file},
                                                        'delay_model_spec': {'file': delay_model_file},
                                                        'props_model_spec': {'file': props_model_file}}}]}

def export_manip_config(manip_config, config_path, print_cmd=False, circuit_name=None, output_base_path=None, N_parallel=None):
    """Writes manipulation config to .json config file(s)."""
    if not isinstance(config_path, list):
        config_path = [config_path]

    fn = f'manip_config__{manip_config["manip"]["name"]}.json'
    for cpath in config_path:
        with open(os.path.join(cpath, fn), 'w') as f:
            json.dump(manip_config, f, indent=2)
        print(f"Config file {fn} written to {cpath}")

    if print_cmd:
        assert circuit_name is not None and output_base_path is not None and N_parallel is not None, \
            "ERROR: circuit_name/output_base_path/N_parallel required for printing launch command!"
        print()
        output_dir = os.path.join(output_base_path, circuit_name + f'__{manip_config["manip"]["name"]}')
        print_launch_cmd(cpath, fn, output_dir, N_parallel)

def print_launch_cmd(config_path, config_fn, output_dir, N_parallel):
    run_cmd = f'sbatch run_rewiring_parallel.sh "{config_fn}" "{output_dir}" {N_parallel}'
    print(f"# LAUNCH COMMAND: [DON'T LAUNCH FROM WITHIN ANOTHER SLURM ALLOCATION!]")
    print(f"cd {config_path}")
    print(run_cmd)


In [45]:
# Model locations (MC2 column models!!)
delay_model_file = os.path.join('/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/model_building', circuit_name, 'model', 'DistDepDelayO1v5mc2EE.json')
props_model_file = os.path.join('/gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/simplified_connectome_models/model_building', circuit_name, 'model', 'ConnPropsPerPathwayO1v5mc2EE.json')
prob_model_file = adj_model_file

In [46]:
manip_config = default_manip_config(circuit_config, seed=3210)
add_manip_to_config(manip_config, adj_name, prob_model_file, delay_model_file, props_model_file)
export_manip_config(manip_config, config_paths, print_cmd=True, circuit_name=circuit_name, output_base_path=output_base_path, N_parallel=500)

Config file manip_config__ConnAddRC_mc2EE_control0_4x.json written to ../configs
Config file manip_config__ConnAddRC_mc2EE_control0_4x.json written to /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/configs

# LAUNCH COMMAND: [DON'T LAUNCH FROM WITHIN ANOTHER SLURM ALLOCATION!]
cd /gpfs/bbp.cscs.ch/project/proj9/bisimplices/pokorny/reciprocal_addition/configs
sbatch run_rewiring_parallel.sh "manip_config__ConnAddRC_mc2EE_control0_4x.json" "/gpfs/bbp.cscs.ch/project/proj9/bisimplices/circuits/O1v5-SONATA__ConnAddRC_mc2EE_control0_4x" 500
